<a href="https://colab.research.google.com/github/jx-dohwan/BERT-FP_MRS/blob/main/fine_tuning/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# fine-tuning

In [1]:

!pip install torch
!pip install transformers==4.25.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.1 MB/s eta 0:00:00


In [2]:

!git clone https://github.com/smilegate-ai/korean_smile_style_dataset
# https://corpus.korean.go.kr/
# https://aihub.or.kr/aihub-data/natural-language/about
     

Cloning into 'korean_smile_style_dataset'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 81 (delta 21), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (81/81), 2.57 MiB | 4.67 MiB/s, done.


In [3]:
import pandas as pd
df = pd.read_csv('/content/korean_smile_style_dataset/smilestyle_dataset.tsv', sep='\t')
df.to_csv('./korean_smile_style_dataset/smile.csv', index=False)

In [4]:
!mv /content/korean_smile_style_dataset/smile.csv /content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset

## 학습/검증 분할하기|

In [5]:
import csv
f = open('/content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset/smile.csv', 'r')
rdr = csv.reader(f)

""" 세션 데이터 저장할 것 """
session_dataset = []
session = []
all_utts = []

""" 실제 데이터 저장 방식 """
for i, line in enumerate(rdr):
    if i == 0:
        header = line
    else:
        utt = line[0]
        if utt.strip() != '':
            all_utts.append(utt)
            session.append(utt)
        else:
            """ 세션 데이터 저장 """
            session_dataset.append(session)
            session = []
""" 마지막 세션 저장 """
session_dataset.append(session)
f.close()

In [6]:
!head -20 /content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset/smile.csv

formal,informal,android,azae,chat,choding,emoticon,enfp,gentle,halbae,halmae,joongding,king,naruto,seonbi,sosim,translator
안녕하세요. 저는 고양이 6마리 키워요.,안녕! 나는 고양이 6마리 키워.,휴먼. 반갑다. 안드로이드는. 고양이. 6마리. 소유중.,아이고 안녕하십니까~ 나는 그냥 고양이 6마리 키우고 있는 사람이여,하잉ㅋㅋ 나 떼걸룩 6마리 키운다!,ㅎㅇ 나 주인님 6마리 모심 ㅋㅋ,안녕!! >< 나는 고양이😺를 ➏ 마리 키우고있어!! 0_0,안녕안녕~! 나 고양이 6마리나 키운다? 완전 대박이징~,"안녕하십니까,, 저는 고양이 6마리 키웁니다.",안녕하신가~... 난 지금 고양이를 6마리 키우고 있다네,하유 시벌것 괭이놈 6마리 키우는데 힘들어 죽겟네,안녕하냐 ㅡㅡ 나 씹냥이 6마리나 키운다 하;,반갑소. 짐은 고양이를 6마리나 키우오.,안녕하냐니깐! 난 고양이를 6마리 키우고있다니깐!,안녕하시오! 소인은 고양이를 6마리 키우고 있소!,안녕… 난 고양이 6마리 키워 ㅠㅠ,반가운. 나는 6마리의 고양이를 소지하고 있다.
고양이를 6마리나요? 키우는거 안 힘드세요?,고양이를 6마리나? 키우는거 안 힘들어?,고양이. 6마리. 양육. 번거로운가.,아니 무슨 고양이를 6마리나? 거 키우는 거 안 힘든가?,엥? 6마리나? 안힘듬?ㅋㅋㅋㅋ,6마리? 에바아니냐 안 힘듦?,고양이를 6마리나?!! w(ﾟДﾟ)w 키우는거 안 힘듬?? (⊙_⊙;),고양이를 6마리나? 완전 대박~ 키우는 거 안 힘들어?!,"고양이를 6마리나 키우십니까? 안 힘드신지,,",고양이를 6마리나? 키우는거 힘들지 않는가?,니기럴 털만 날리는 거 키우기 안 힘들데?,아니 고양이를 6마리나? 안힘드냐?,고양이를 6마리나? 키우는게 수고스럽진 않소?,고양이를 6마리나? 키우는거 힘들지 않냐니깐?,고양이를 6마리나 키우고 있는 것이오? 힘들지 않소?,고양이..6마리나? ㅠ 키우는건 혹시 안힘들어..?,6마리의

In [7]:
print(len(session_dataset))


236


In [8]:
train_session = session_dataset[:-15]
valid_session = session_dataset[-15:]
len(train_session), len(valid_session)

(221, 15)

In [9]:
train_session[0]
# 첫번째 데이터
# context: 안녕하세요. 저는 고양이 6마리 키워요.
# response: 고양이를 6마리나요? 키우는거 안 힘드세요?

# 두번째 데이터
# context: 안녕하세요. 저는 고양이 6마리 키워요. | 고양이를 6마리나요? 키우는거 안 힘드세요?
# response: 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.

# 세번째 데이터
# context: 안녕하세요. 저는 고양이 6마리 키워요. | 고양이를 6마리나요? 키우는거 안 힘드세요? | 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.
# response: 가장 나이가 많은 고양이가 어떻게 돼요?

['안녕하세요. 저는 고양이 6마리 키워요.',
 '고양이를 6마리나요? 키우는거 안 힘드세요?',
 '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
 '가장 나이가 많은 고양이가 어떻게 돼요?',
 '여섯 살입니다. 갈색 고양이에요.',
 '그럼 가장 어린 고양이가 어떻게 돼요?',
 '한 살입니다. 작년에 분양 받았어요.',
 '그럼 고양이들끼리 안 싸우나요?',
 '저희 일곱은 다같이 한 가족입니다. 싸우는 일은 없어요.']

## positive / negative pairs 구성

In [10]:
from collections import defaultdict
train_json = defaultdict(dict)
count = 0
for session in train_session:
    context = [session[0]]
    for turn in range(1, len(session)):
        utt = session[turn] 
        train_json[count]['context'] = context[:]
        train_json[count]['response'] = utt
        train_json[count]['label'] = 1
        context.append(utt)
        count += 1
    break

In [11]:
train_json

defaultdict(dict,
            {0: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.'],
              'response': '고양이를 6마리나요? 키우는거 안 힘드세요?',
              'label': 1},
             1: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.',
               '고양이를 6마리나요? 키우는거 안 힘드세요?'],
              'response': '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
              'label': 1},
             2: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.',
               '고양이를 6마리나요? 키우는거 안 힘드세요?',
               '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.'],
              'response': '가장 나이가 많은 고양이가 어떻게 돼요?',
              'label': 1},
             3: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.',
               '고양이를 6마리나요? 키우는거 안 힘드세요?',
               '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
               '가장 나이가 많은 고양이가 어떻게 돼요?'],
              'response': '여섯 살입니다. 갈색 고양이에요.',
              'label': 1},
             4: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.',
               '고양이를 6마리나요? 키우는거 안 힘드세요?',
               '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
               '가장 나이가 많은 고양이가 어떻

In [12]:
import random
use_turns = 5 # 꼭 5일 필요는 없음
neg_nums = 4 # 꼭 4일 필요는 없음

from collections import defaultdict
train_json = defaultdict(dict)
count = 0
for session in train_session:
    context = [session[0]]
    for turn in range(1, len(session)):
        utt = session[turn]
        train_json[count]['context'] = context[:][-use_turns:]
        train_json[count]['positive_response'] = utt
        context.append(utt)

        negative_candidates = random.sample(all_utts, neg_nums)
        train_json[count]['negative_responses'] = negative_candidates
        count += 1
    break

In [13]:
train_json

defaultdict(dict,
            {0: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.'],
              'positive_response': '고양이를 6마리나요? 키우는거 안 힘드세요?',
              'negative_responses': ['어떤 문제가 있으신가요?',
               '아이들이 책 읽는 걸 좋아하나요?',
               '그리고 바나나는 영양도 풍부한 것 같아요.',
               '부는 하데스가 관장하는 영역이 아닌가요?']},
             1: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.',
               '고양이를 6마리나요? 키우는거 안 힘드세요?'],
              'positive_response': '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
              'negative_responses': ['학생이라고 하시니 추억들이 생각나네요. 힘들지만 열심히 노력하시길 바랄게요.',
               '그런 논리라면 저는 너무 많은 음식을 싫어하는 것이 되겠네요.',
               '그랬으면 좋겠네요. 하지만 요즘은 일이 별로 없어요.',
               '두시간이면 갈 길을 네시간이나 가셨겠네요.']},
             2: {'context': ['안녕하세요. 저는 고양이 6마리 키워요.',
               '고양이를 6마리나요? 키우는거 안 힘드세요?',
               '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.'],
              'positive_response': '가장 나이가 많은 고양이가 어떻게 돼요?',
              'negative_responses': ['네, 피아노를 칠 줄 압니다.',
               '저는 주로 테니스 경기를 봅

In [14]:
# import random
# neg_nums = 4

# from collections import defaultdict
# train_json = defaultdict(dict)
# count = 0
# for session in train_session:
#     context = [session[0]]
#     for turn in range(1, len(session)):
#         utt = session[turn]
#         train_json[count]['context'] = context[:][-use_turns:]
#         train_json[count]['positive_response'] = utt        
#         context.append(utt)        
        
#         negative_candidates = random.sample(all_utts, neg_nums)
#         train_json[count]['negative_responses'] = negative_candidates
#         count += 1
    
# import json
# with open('/content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset/train.json', 'w') as outfile:
#     json.dump(train_json, outfile)

In [15]:
import json, pdb
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class fine_loader(Dataset):
    def __init__(self, data_path):
        self.tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
        special_tokens = {'sep_token': '<SEP>'}
        self.tokenizer.add_special_tokens(special_tokens)
        
        """ 세션 데이터 """
        with open(data_path, 'r') as f:
            self.session_dataset = json.load(f)
        
    def __len__(self): # 기본적인 구성
        return len(self.session_dataset)
    
    def __getitem__(self, idx): # 기본적인 구성
        session = self.session_dataset[str(idx)]
        context = session['context']
        positive_response = session['positive_response']
        negative_responses = session['negative_responses']
        session_tokens = []
        session_labels = []
        
        """ MRS 입력 """
        context_token = [self.tokenizer.cls_token_id]
        for utt in context:
            context_token += self.tokenizer.encode(utt, add_special_tokens=False)
            context_token += [self.tokenizer.sep_token_id]
        
        pos_respons_token = [self.tokenizer.eos_token_id]
        pos_respons_token += self.tokenizer.encode(positive_response, add_special_tokens=False)
        positive_tokens = context_token + pos_respons_token
        session_tokens.append(positive_tokens)
        session_labels.append(1)
        
        for negative_response in negative_responses:
            neg_respons_token = [self.tokenizer.eos_token_id]
            neg_respons_token += self.tokenizer.encode(negative_response, add_special_tokens=False)
            negative_tokens = context_token + neg_respons_token        
            session_tokens.append(negative_tokens)
            session_labels.append(0)
        
        return session_tokens, session_labels

In [16]:
from torch.utils.data import DataLoader
train_path = '/content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset/train.json'
train_dataset = fine_loader(train_path)

In [17]:
session_tokens, session_labels = train_dataset[1]

In [18]:
len(session_tokens), len(session_tokens[0]), len(session_tokens[1])

(5, 44, 39)

In [19]:
train_dataset.tokenizer.decode(session_tokens[4])

'[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> [SEP] 저는 항상 옷을 같은 곳에서 삽니다. 직접 만들지는 않고요.'

In [20]:
session_labels

[1, 0, 0, 0, 0]

## 배치 처리하기

In [21]:
import json, pdb
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import torch

class fine_loader(Dataset):
    def __init__(self, data_path):
        self.tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
        special_tokens = {'sep_token': '<SEP>'}
        self.tokenizer.add_special_tokens(special_tokens)
        
        """ 세션 데이터 """
        with open(data_path, 'r') as f:
            self.session_dataset = json.load(f)
        
    def __len__(self): # 기본적인 구성
        return len(self.session_dataset)
    
    def __getitem__(self, idx): # 기본적인 구성
        session = self.session_dataset[str(idx)]
        context = session['context']
        positive_response = session['positive_response']
        negative_responses = session['negative_responses']
        session_tokens = []
        session_labels = []
        
        """ MRS 입력 """
        context_token = [self.tokenizer.cls_token_id]
        for utt in context:
            context_token += self.tokenizer.encode(utt, add_special_tokens=False)
            context_token += [self.tokenizer.sep_token_id]
        
        pos_respons_token = [self.tokenizer.eos_token_id]
        pos_respons_token += self.tokenizer.encode(positive_response, add_special_tokens=False)
        positive_tokens = context_token + pos_respons_token
        session_tokens.append(positive_tokens)
        session_labels.append(1)
        
        for negative_response in negative_responses:
            neg_respons_token = [self.tokenizer.eos_token_id]
            neg_respons_token += self.tokenizer.encode(negative_response, add_special_tokens=False)
            negative_tokens = context_token + neg_respons_token        
            session_tokens.append(negative_tokens)
            session_labels.append(0)
        
        return session_tokens, session_labels
    
    def collate_fn(self, sessions): # 배치를 위한 구성
        '''
            input:
                data: [(session1), (session2), ... ]
            return:
                batch_input_tokens_pad: (B, L) padded
                batch_labels: (B)
        '''        
        # 최대 길이 찾기 for padding
        max_input_len = 0
        for session in sessions:
            session_tokens, session_labels = session
            input_tokens_len = [len(x) for x in session_tokens]
            max_input_len = max(max_input_len, max(input_tokens_len))
        
        batch_input_tokens, batch_input_labels = [], []
        batch_input_attentions = []
        for session in sessions:
            session_tokens, session_labels = session
            for session_token in session_tokens:
                input_token = session_token + [self.tokenizer.pad_token_id for _ in range(max_input_len-len(session_token))]
                input_attention = [1 for _ in range(len(session_token))] + [0 for _ in range(max_input_len-len(session_token))]
                batch_input_tokens.append(input_token)
                batch_input_attentions.append(input_attention)
            batch_input_labels += session_labels
        
        return torch.tensor(batch_input_tokens), torch.tensor(batch_input_attentions), torch.tensor(batch_input_labels)

In [22]:
!touch dataset.py

In [23]:
from dataset import fine_loader
from torch.utils.data import DataLoader
train_path = '/content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset/train.json'
train_dataset = fine_loader(train_path)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=False, num_workers=4, collate_fn=train_dataset.collate_fn)

In [24]:
for data in train_dataloader:
    batch_input_tokens, batch_input_attentions, batch_input_labels = data
    break

In [25]:
batch_input_tokens.shape, batch_input_attentions.shape, batch_input_labels.shape

(torch.Size([10, 53]), torch.Size([10, 53]), torch.Size([10]))

In [26]:
batch_input_tokens

tensor([[    0,  5891,  2205,  5971,    18,  1535,  2259,  7003,    26, 12736,
          5972,  2182,    18, 32000,     2,  7003,  2138,    26, 12736,  2075,
          2182,    35,  5688,  2259,  2180,  1378, 18941,  5971,    35,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1],
        [    0,  5891,  2205,  5971,    18,  1535,  2259,  7003,    26, 12736,
          5972,  2182,    18, 32000,     2, 10073, 22087,  2570,  2079,  5306,
          2256,  2955,  2069,  4920,  5723, 11800,    18,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1],
        [    0,  5891,  2205,  5971,    18,  1535,  2259,  7003,    26, 12736,
          5972,  2182,    18, 32000,     2,  4177,  4241,  2052,  597

## 모델링하기

In [27]:
from transformers import RobertaForMaskedLM, AutoTokenizer
import torch
import torch.nn as nn

class FineModel(nn.Module):
    def __init__(self):
        super(FineModel, self).__init__()
        self.model = RobertaForMaskedLM.from_pretrained('klue/roberta-base')
        self.hiddenDim = self.model.config.hidden_size
        
        self.tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
        special_tokens = {'sep_token': '<SEP>'}
        self.tokenizer.add_special_tokens(special_tokens)
        self.model.resize_token_embeddings(len(self.tokenizer))        
        
        """ score matrix """
        # self.W = nn.Linear(self.hiddenDim, 3)
        self.W2 = nn.Linear(self.hiddenDim, 2)
        
    def forward(self, batch_input_tokens, batch_input_attentions):
        ## Binary Classification (pointwise)
        outputs = self.model(batch_input_tokens, attention_mask=batch_input_attentions, output_hidden_states=True)['hidden_states'][-1] # [B, L, hidden_dim]
        cls_outputs = outputs[:,0,:] # [B, hidden_dim]/0은 cls를 해당
        cls_logits = self.W2(cls_outputs) # [B, 2]
         
        return cls_logits

In [28]:
!touch model.py

In [29]:
import torch
from model import FineModel
fine_model = FineModel()
result = fine_model(batch_input_tokens, batch_input_attentions)

In [30]:
result

tensor([[ 0.3844, -0.1820],
        [ 0.4085, -0.1985],
        [ 0.4116, -0.1793],
        [ 0.4070, -0.1682],
        [ 0.3877, -0.1602],
        [ 0.3821, -0.2148],
        [ 0.3955, -0.1609],
        [ 0.4285, -0.2139],
        [ 0.4197, -0.1930],
        [ 0.4493, -0.1887]], grad_fn=<AddmmBackward0>)

In [31]:
# fine_model.load_state_dict(torch.load('/content/drive/MyDrive/인공지능/멀티턴응답선택/post_model.bin'))
# result2 = fine_model(batch_input_tokens, batch_input_attentions)


In [32]:
# key가 안맞는 부분은 초기화 사용
# fine_model = FineModel()
fine_model.load_state_dict(torch.load('/content/drive/MyDrive/인공지능/멀티턴응답선택/post_model.bin'), strict=False)

_IncompatibleKeys(missing_keys=['W2.weight', 'W2.bias'], unexpected_keys=['W.weight', 'W.bias'])

In [33]:
result2 = fine_model(batch_input_tokens, batch_input_attentions)
result2

tensor([[ 0.1197, -0.1944],
        [ 0.1042,  0.0373],
        [-0.1418, -0.1794],
        [ 0.1040, -0.0032],
        [-0.2196, -0.2352],
        [ 0.1223, -0.1943],
        [ 0.1872, -0.0718],
        [ 0.2275, -0.0715],
        [ 0.3380, -0.0444],
        [ 0.2661,  0.0124]], grad_fn=<AddmmBackward0>)

## 학습코드

In [34]:
from dataset import fine_loader
from torch.utils.data import DataLoader
train_path ='/content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset/train.json'
train_dataset = fine_loader(train_path)

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=4, collate_fn=train_dataset.collate_fn)

dev_path = '/content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset/dev.json'
dev_dataset = fine_loader(dev_path)
dev_dataloader = DataLoader(dev_dataset, batch_size=1, shuffle=False, num_workers=4, collate_fn=dev_dataset.collate_fn)

In [35]:
len(train_dataloader), len(dev_dataloader)

(3020, 214)

In [36]:
import torch
from model import FineModel
fine_model = FineModel().cuda()
fine_model.load_state_dict(torch.load('/content/drive/MyDrive/인공지능/멀티턴응답선택/post_model.bin'), strict=False)

_IncompatibleKeys(missing_keys=['W2.weight', 'W2.bias'], unexpected_keys=['W.weight', 'W.bias'])

In [37]:
import torch.nn as nn
def CELoss(pred_outs, labels):
    """
        pred_outs : [batch, clsNum]
        labels : [batch]
    """
    loss = nn.CrossEntropyLoss()
    loss_val = loss(pred_outs, labels)
    return loss_val

In [38]:
import torch
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import pdb

training_epochs = 5
max_grad_norm = 10
lr = 1e-5
num_training_steps = len(train_dataset)*training_epochs
num_warmup_steps = len(train_dataset)
optimizer = torch.optim.AdamW(fine_model.parameters(), lr=lr) # , eps=1e-06, weight_decay=0.01
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

for epoch in range(training_epochs):
    fine_model.train() 
    for i_batch, data in enumerate(tqdm(train_dataloader)):
        batch_input_tokens, batch_input_attentions, batch_input_labels = data
        
        batch_input_tokens = batch_input_tokens.cuda()
        batch_input_attentions = batch_input_attentions.cuda()
        batch_input_labels = batch_input_labels.cuda()
        
        """Prediction"""
        outputs = fine_model(batch_input_tokens, batch_input_attentions)
        loss_val = CELoss(outputs, batch_input_labels)
        
        loss_val.backward()
        torch.nn.utils.clip_grad_norm_(fine_model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()        
        break
    break

  0%|          | 0/3020 [00:04<?, ?it/s]


## 평가방법

In [39]:
## label: [1, 0, 0, 1, 0, 0]
## pred:  [1, 0, 1, 0, 1, 0]
# precision = 1/3
# recall = 1/2
# 100명의 입원자가있고 암발생을 검진하는 AI. 실제로는 99명은 암에 안걸렸고 1명이 암걸렸습니다.
# 모델입장에서 모두 암에 안걸렸다고 하면? accuracy = 99%
# recall = 0/1
# 모델입장에서 10명 암에걸렸다고 판단. 이중에는 실제 암환자가 있었다.
# precision = 1/10, recall = 1/1

# 10개중 9개가 negative 1개가 positive
# R10@1: reponse candidate가 10개가 있다. 1이라는 것은 top-1을 보겠다. --> 1/1, 0/1
# R10@5: reponse candidate가 10개가 있다. 5이라는 것은 top-5을 보겠다. --> 1/1, 0/1

# 10개중 7개가 negative 3개가 positve (Douban은 positive 개수가 매번 달라짐)
# R10@1: reponse candidate가 10개가 있다. 1이라는 것은 top-1을 보겠다. --> 1/3, 0/3
# P@1: reponse candidate가 10개가 있다. 1이라는 것은 top-1을 보겠다. --> 1/1, 0/1

# 결과적으로 이 실습에서는 R@1을 사용할겁니다.

In [40]:
import pdb
from torch.nn.functional import softmax
def CalP1(fine_model, dataloader):
    fine_model.eval()
    correct = 0
    for i_batch, data in enumerate(tqdm(dataloader, desc='evaluation')):
        batch_input_tokens, batch_input_attentions, batch_input_labels = data

        batch_input_tokens = batch_input_tokens.cuda()
        batch_input_attentions = batch_input_attentions.cuda()
        batch_input_labels = batch_input_labels.cuda()

        """ Prediction """
        outputs = fine_model(batch_input_tokens, batch_input_attentions)
        probs = softmax(outputs, 1)
        true_probs = probs[:,1] # pisitive에 해당하는 확률률
        pred_ind = true_probs.argmax(0).item() # 5개의 후보중 가장 큰 인덱스
        gt_ind = batch_input_labels.argmax(0).item() # 레이블에서 가장 큰 인덱스 가져오기
        if pred_ind == gt_ind:
            correct += 1
    return round(correct/len(dataloader)*100, 2)

In [41]:
CalP1(fine_model, dev_dataloader)

evaluation: 100%|██████████| 214/214 [00:04<00:00, 51.07it/s]


75.7

In [42]:
import os
def SaveModel(model, path):
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(model.state_dict(), os.path.join(path, '/content/drive/MyDrive/인공지능/멀티턴응답선택/fine_model.bin'))

## 최종 학습 코드

In [43]:
""" 데이터 로딩 """
from dataset import fine_loader
from torch.utils.data import DataLoader
train_path ='/content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset/train.json'
train_dataset = fine_loader(train_path)

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=4, collate_fn=train_dataset.collate_fn)

dev_path = '/content/drive/MyDrive/인공지능/멀티턴응답선택/korean_smile_style_dataset/dev.json'
dev_dataset = fine_loader(dev_path)
dev_dataloader = DataLoader(dev_dataset, batch_size=1, shuffle=False, num_workers=4, collate_fn=dev_dataset.collate_fn)
""" 모델 로딩 """
import torch
from model import FineModel
fine_model = FineModel().cuda()
fine_model.load_state_dict(torch.load('/content/drive/MyDrive/인공지능/멀티턴응답선택/post_model.bin'), strict=False)

""" loss 식 """
import torch.nn as nn
def CELoss(pred_outs, labels):
    """
        pred_outs: [batch, clsNum]
        labels: [batch]
    """
    loss = nn.CrossEntropyLoss()
    loss_val = loss(pred_outs, labels)
    return loss_val

""" 평가 """
from torch.nn.functional import softmax
def CalP1(fine_model, dataloader):
    fine_model.eval()
    correct = 0
    for i_batch, data in enumerate(tqdm(dataloader, desc="evaluation")):
        batch_input_tokens, batch_input_attentions, batch_input_labels = data
        
        batch_input_tokens = batch_input_tokens.cuda()
        batch_input_attentions = batch_input_attentions.cuda()
        batch_input_labels = batch_input_labels.cuda()
        
        """Prediction"""
        outputs = fine_model(batch_input_tokens, batch_input_attentions)    
        probs = softmax(outputs, 1)
        true_probs = probs[:,1]
        pred_ind = true_probs.argmax(0).item()
        gt_ind = batch_input_labels.argmax(0).item()
        
        if pred_ind == gt_ind:
            correct += 1
    return round(correct/len(dataloader)*100, 2)

""" 모델 저장 """
import os
def SaveModel(model, path):
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(model.state_dict(), os.path.join(path, '/content/drive/MyDrive/인공지능/멀티턴응답선택/fine_model.bin'))

""" 학습 """
import torch
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import pdb

training_epochs = 5
max_grad_norm = 10
lr = 1e-6
num_training_steps = len(train_dataset)*training_epochs
num_warmup_steps = len(train_dataset)
optimizer = torch.optim.AdamW(fine_model.parameters(), lr=lr) # , eps=1e-06, weight_decay=0.01
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

best_p1 = 0
for epoch in range(training_epochs):
    fine_model.train() 
    for i_batch, data in enumerate(tqdm(train_dataloader)):
        batch_input_tokens, batch_input_attentions, batch_input_labels = data
        
        batch_input_tokens = batch_input_tokens.cuda()
        batch_input_attentions = batch_input_attentions.cuda()
        batch_input_labels = batch_input_labels.cuda()
        
        """Prediction"""
        outputs = fine_model(batch_input_tokens, batch_input_attentions)
        loss_val = CELoss(outputs, batch_input_labels)
        
        loss_val.backward()
        torch.nn.utils.clip_grad_norm_(fine_model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
    fine_model.eval()
    p1 = CalP1(fine_model, dev_dataloader)
    print(f"Epoch: {epoch}번째 모델 성능(p@1): {p1}")
    if p1 > best_p1:
        best_p1 = p1
        print(f"BEST 성능(p@1): {best_p1}")
        SaveModel(fine_model, '.')

evaluation: 100%|██████████| 214/214 [00:03<00:00, 55.54it/s]


Epoch: 0번째 모델 성능(p@1): 93.46
BEST 성능(p@1): 93.46


evaluation: 100%|██████████| 214/214 [00:03<00:00, 54.71it/s]


Epoch: 1번째 모델 성능(p@1): 94.86
BEST 성능(p@1): 94.86


evaluation: 100%|██████████| 214/214 [00:03<00:00, 55.18it/s]


Epoch: 2번째 모델 성능(p@1): 94.39


evaluation: 100%|██████████| 214/214 [00:03<00:00, 55.50it/s]


Epoch: 3번째 모델 성능(p@1): 94.39


evaluation: 100%|██████████| 214/214 [00:03<00:00, 55.11it/s]

Epoch: 4번째 모델 성능(p@1): 94.86


In [44]:
!touch fine_train.py

In [45]:
# !python3 fine_train.py

# 멀티턴 응답선택 모델 분석하기

In [47]:
# # 모델 성능 상펴보기 (R5@1)
# ##############################################
# Post-training X
# BEST 성능(p@1): 91.1214953271028
# ##############################################
# Post-training O
# BEST 성능(p@1): 93.92523364485982

In [ ]:
# 테스트 샘플이 랜덤 네거티브로만 샘플링되었기 때문에 비교적 쉬운 상황이라 성능이 높음
# positive가 여러 개인 후보일 경우는 P@1 등의 다른 평가메트릭도 같이 보는게 좋음

## 모델 테스트하기 - 자신만의 입력으로 테스트해보기

In [ ]:
# !cp './drive/MyDrive/공부자료/03_패캠교육자료/fine_model.bin' .

In [48]:
""" 모델 로딩 """
import torch
from model import FineModel
fine_model = FineModel().cuda()
fine_model.load_state_dict(torch.load('/content/drive/MyDrive/인공지능/멀티턴응답선택/fine_model.bin'))

<All keys matched successfully>

In [49]:
# 모델이 학습되는 데이터가 너무 적어서 다양한 입력 형태를 반영하기 어렵다.
# 컨텍스트는 dev 셋에서 하나의 샘플을 가져옴
context = ["어떤 문제가 있으신가요?", "어떤 차를 사야 할지 잘 모르겠어요.", "차는 한 번 사면 10년도 넘게 써서, 신중하게 골라야 해요."]
candidates = ["저 좀 도와주실 수 있나요?", "저는 농구를 좋아합니다.", "자동차는 신중히 골라야합니다.", "저는 차 마시는거를 좋아합니다.", "날씨가 화창합니다.", "저는 차를 좋아합니다."]

In [50]:
context_token = [fine_model.tokenizer.cls_token_id]
for utt in context:
    context_token += fine_model.tokenizer.encode(utt, add_special_tokens=False)
    context_token += [fine_model.tokenizer.sep_token_id]

session_tokens = []    
for response in candidates:
    response_token = [fine_model.tokenizer.eos_token_id]
    response_token += fine_model.tokenizer.encode(response, add_special_tokens=False)
    candidate_tokens = context_token + response_token        
    session_tokens.append(candidate_tokens)
    
# 최대 길이 찾기 for padding
max_input_len = 0
input_tokens_len = [len(x) for x in session_tokens]
max_input_len = max(max_input_len, max(input_tokens_len))    
    
batch_input_tokens = []
batch_input_attentions = []
for session_token in session_tokens:
    input_token = session_token + [fine_model.tokenizer.pad_token_id for _ in range(max_input_len-len(session_token))]
    input_attention = [1 for _ in range(len(session_token))] + [0 for _ in range(max_input_len-len(session_token))]
    batch_input_tokens.append(input_token)
    batch_input_attentions.append(input_attention)
    
batch_input_tokens = torch.tensor(batch_input_tokens).cuda()
batch_input_attentions = torch.tensor(batch_input_attentions).cuda()

In [51]:
batch_input_tokens.shape, batch_input_attentions.shape

(torch.Size([6, 53]), torch.Size([6, 53]))

In [52]:
softmax = torch.nn.Softmax(dim=1)
results = fine_model(batch_input_tokens, batch_input_attentions)
prob = softmax(results)
true_prob = prob[:,1].tolist()

print(context)
for utt, prob in zip(candidates, true_prob):
    print(utt, '##', round(prob,3))

['어떤 문제가 있으신가요?', '어떤 차를 사야 할지 잘 모르겠어요.', '차는 한 번 사면 10년도 넘게 써서, 신중하게 골라야 해요.']
저 좀 도와주실 수 있나요? ## 0.003
저는 농구를 좋아합니다. ## 0.001
자동차는 신중히 골라야합니다. ## 0.967
저는 차 마시는거를 좋아합니다. ## 0.957
날씨가 화창합니다. ## 0.003
저는 차를 좋아합니다. ## 0.976


## 모델링의 발전 생각해보기

In [53]:
## 학습 데이터가 현저히 부족하기 때문에 실제 다양한 테스트에 대해 잘 작동하기 어렵다.
## 이 테스크는 기본적으로 엄청나게 많은 학습 데이터를 사용하는 것이 일반적 (몇십만개 정도)
# ㄴ 이런 것을 해결하기 위해서는 멀티지피유 학습
## 학습 데이터와 유사한 톤을 사용하는 것이 좋다.
# ㄴ 줄임말, 그 시대만의 용어?
## 키워드에따라 bias가 되는 문제는 발생 (이를 해결하기 위한 연구도 존재)
#--------------------------------------------------------------------------------#
## 실제 화자가 2명만 존재하는 시나리오일까? (3명이상의 다화자 발화인 경우에는?)
# ㄴ [CLS] <s1> 발화1 <s2> 발화2 <s1> 발화3 <s3> [SEP] 발화4(candidate)
## 현재는 정적 네거티브 샘플링 --> 동적으로 샘플링 or negative 후보 생성하기
## 다른 도메인으로의 확장은?
## 그래프적인 모델링
## 외부지식, 즉 발화의 추가적인 특징을 (토픽, 질문인지 평서문인지 등) 활용할 수 있다면?
## 적당한 context 길이를 선정 / 혹은 같은 세션을 판단할 수 있는 모듈이 있다면?
## distillation (teacher: 미래의 발화를 활용한 모델)